# RScript-Bridge

> Bridge between Stactics AICore framework and RScript prediction scripts

In [ ]:
#| default_exp rscript

### Some things to set up first

Notebooks use nbdev thingses and `addroot` makes importing from
the repo-directory more convenient.

In [ ]:
from nbdev.showdoc import *
import logging
#import addroot

In [ ]:
logging.basicConfig(level=logging.DEBUG)

In [ ]:
#| export

import json, os, fcntl, logging
import subprocess
import hashlib
from collections import namedtuple

from corebridge.aicorebridge import AICoreModule

Loading corebridge.aicorebridge 0.2.17 from /home/fenke/repos/corebridge/corebridge/aicorebridge.py


AICore uses an `assets` dir from which we can read files, like scripts
and a `save` dir were modules can write and read files.

In [ ]:
#| export

syslog = logging.getLogger(__name__)

In [ ]:

assets_dir = os.path.join(os.path.abspath(os.getcwd()), 'assets', 'rscript')
save_dir = os.path.join(os.path.abspath(os.getcwd()), 'saves', 'rscript')


In [ ]:
#| exports

def get_asset_path(script_name, assets_dir:str): 
    return os.path.join(assets_dir, script_name)
def get_save_path(datafile_name, save_dir): 
    return os.path.join(save_dir, datafile_name)


In [ ]:
#| hide
from tabulate import tabulate

In [ ]:
#| hide
def markdown_table(feature_dict):
    return Markdown(tabulate(
      [[v for v in row.values()] for row in feature_dict],
      headers=[k for k in feature_dict[0].keys()],
        tablefmt='github'
    ))

def markdown_flow_table(flow_table):
    columns = set([C for S in flow_table for C in flow_table[S]])
    print(columns)
    return markdown_table()(
        [
            dict(file=fn, **{c:rd.get(c,' ') for c in columns}) 
            for fn, rd in flow_table.items()
        ]    
    )

In [ ]:
#| hide

from IPython.display import display
from IPython.display import Markdown


In [ ]:
#| hide
def display_table(feature_dict):
    display(markdown_table(feature_dict))
    
data_file_flow = {}

def display_flow_table(flow_table):
    columns = set([C for S in data_file_flow for C in data_file_flow[S]])
    print(columns)
    display(markdown_table(        [
            dict(file=fn, **{c:rd.get(c,' ') for c in columns}) 
            for fn, rd in flow_table.items()
        ]    
    ))


## Running R code

Scripts written in R can be run from a Python program using `subprocess` and `Rscript`.



### `Rscript`

A script can be run from the commandline with

    Rscript ascript.R



### `subproces`

[Python's `subprocess`module](https://docs.python.org/3.11/library/subprocess.html#) has the tools to execute external programs like `Rscript`


In [ ]:

subprocess.run(['Rscript',get_asset_path('hello.R', assets_dir)], capture_output=True).stdout.decode('UTF-8')

'[1] "hello world"\n'

## Example: sapflow prediction scripts



### `Data_preparation.R`

#### Libraries

* lubridate
* stringer
* zoo

#### Input

* `Data/Meta_data.csv`
* `Data/Sapflux_Tilia_train.csv`
* `Data/Weather_Tilia_train.csv`
* `Data/Weather_Tilia_pred.csv`

#### Output

* `Modelling_data.RData`
* `Prediction_data.RData`


In [ ]:
data_file_flow['Data_preparation.R'] = {
      "in": [
         "Data/Meta_data.csv",
         "Data/Sapflux_Tilia_train.csv",
         "Data/Weather_Tilia_train.csv",
         "Data/Weather_Tilia_pred.csv"
      ],
      "out": [
         "Modelling_data.RData",
         "Prediction_data.RData"
      ],
    'libs':['lubridate', 'stringr', 'zoo']
}


### `Prediction_part1.R`

#### Libraries

* lubridate
* stringr
* mgcv

#### Input

* `Modelling_data.RData`

#### Output

* `Fitted_models.RData`
* `Weights.RData`



In [ ]:
data_file_flow['Prediction_part1.R'] = {
      "in": [
         "Modelling_data.RData"
      ],
      "out": [
         "Fitted_models.RData",
         "Weights.RData"
      ],
    'libs':['lubridate', 'stringr', 'mgcv']
}



### `Prediction_part2.R`

#### Libraries

* lubridate
* stringr
* mgcv

#### Input

* `Fitted_models.RData`
* `Weights.RData`
* `Modelling_data.RData`
* `Prediction_data.RData`

#### Output

* `Predicted_sapflux.RData`

In [ ]:
data_file_flow['Prediction_part2.R'] = {
    "in":[
        'Fitted_models.RData',
        'Weights.RData',
        'Modelling_data.RData',
        'Prediction_data.RData'
    ],
    "out":[
        'Predicted_sapflux.RData'
    ],
    'libs':['lubridate', 'stringr', 'mgcv']
}


### `Prediction_part3.R`

#### Libraries

* lubridate
* stringr

#### Input

* `Predicted_sapflux.RData`

#### Output

* `Predicted_water_usage.RData`


In [ ]:
data_file_flow['Prediction_part3.R'] = {
    'in':['Predicted_sapflux.RData'],
    'out':['Predicted_water_usage.RData'],
    'libs':['lubridate', 'stringr']
}

In [ ]:
#| hide
from functools import reduce


In [ ]:
#| hide
script_order = dict(zip(data_file_flow.keys(), range(len(data_file_flow.keys()))))

# add the name to the objects
data_file_flow = {
    script_order[k]:{**v, 'name':k}
    for k,v in data_file_flow.items()
}


data_files = reduce(
    lambda Y,X:Y if (X in Y) else [*Y,X],
    [
        f
        for S,P in data_file_flow.items() # patterns
        for D,F in P.items()
        for f in F
        if D in ['in','out']
        
    ],
    []
)
display(Markdown(tabulate(
    [
        [F.split('/')[-1]]+[
            'in' if F in P['in'] else 'out' if F in P['out'] else '--' 
            for S,P in data_file_flow.items()
        ] 
        for F in data_files
    ],
    headers=['data-file / script'] + [I['name'] for I in data_file_flow.values()],
    tablefmt='github'
)))

| data-file / script          | Data_preparation.R   | Prediction_part1.R   | Prediction_part2.R   | Prediction_part3.R   |
|-----------------------------|----------------------|----------------------|----------------------|----------------------|
| Meta_data.csv               | in                   | --                   | --                   | --                   |
| Sapflux_Tilia_train.csv     | in                   | --                   | --                   | --                   |
| Weather_Tilia_train.csv     | in                   | --                   | --                   | --                   |
| Weather_Tilia_pred.csv      | in                   | --                   | --                   | --                   |
| Modelling_data.RData        | out                  | in                   | in                   | --                   |
| Prediction_data.RData       | out                  | --                   | in                   | --                   |
| Fitted_models.RData         | --                   | out                  | in                   | --                   |
| Weights.RData               | --                   | out                  | in                   | --                   |
| Predicted_sapflux.RData     | --                   | --                   | out                  | in                   |
| Predicted_water_usage.RData | --                   | --                   | --                   | out                  |

In [ ]:
data_files

['Data/Meta_data.csv',
 'Data/Sapflux_Tilia_train.csv',
 'Data/Weather_Tilia_train.csv',
 'Data/Weather_Tilia_pred.csv',
 'Modelling_data.RData',
 'Prediction_data.RData',
 'Fitted_models.RData',
 'Weights.RData',
 'Predicted_sapflux.RData',
 'Predicted_water_usage.RData']

In [ ]:
data_file_flow

{0: {'in': ['Data/Meta_data.csv',
   'Data/Sapflux_Tilia_train.csv',
   'Data/Weather_Tilia_train.csv',
   'Data/Weather_Tilia_pred.csv'],
  'out': ['Modelling_data.RData', 'Prediction_data.RData'],
  'libs': ['lubridate', 'stringr', 'zoo'],
  'name': 'Data_preparation.R'},
 1: {'in': ['Modelling_data.RData'],
  'out': ['Fitted_models.RData', 'Weights.RData'],
  'libs': ['lubridate', 'stringr', 'mgcv'],
  'name': 'Prediction_part1.R'},
 2: {'in': ['Fitted_models.RData',
   'Weights.RData',
   'Modelling_data.RData',
   'Prediction_data.RData'],
  'out': ['Predicted_sapflux.RData'],
  'libs': ['lubridate', 'stringr', 'mgcv'],
  'name': 'Prediction_part2.R'},
 3: {'in': ['Predicted_sapflux.RData'],
  'out': ['Predicted_water_usage.RData'],
  'libs': ['lubridate', 'stringr'],
  'name': 'Prediction_part3.R'}}

## Import R libraries

Importing libraries can be done with

    Rscript -e 'install.packages("drat", repos="https://cloud.r-project.org")'

In [ ]:
print(subprocess.run(['Rscript','--version', ], capture_output=True).stdout.decode('UTF-8'))


Rscript (R) version 4.2.2 (2022-10-31)



In [ ]:
rversion = subprocess.run(['Rscript','--version', ], capture_output=True)
print(rversion.stdout.decode('UTF-8'))

Rscript (R) version 4.2.2 (2022-10-31)



In [ ]:
list(set([L for V in data_file_flow.values() for L in V['libs']]))

['zoo', 'lubridate', 'mgcv', 'stringr']

In [ ]:
run_script_result = subprocess.run(['Rscript','-e', "library(mgcv)"], capture_output=True)
print(run_script_result.stderr.decode('UTF-8'))

Loading required package: nlme
This is mgcv 1.9-1. For overview type 'help("mgcv-package")'.



In [ ]:
#| exports

def install_R_package(pkg:str|list):
    """
    Checks and if neccesary installs an R package

    Parameters
    ----------
    pkg : str|list
        name(s) of the package(s)
    """

    if isinstance(pkg, str):
        pkg = [pkg]

    for pkg_i in pkg:
        print(f"Installing package {pkg_i} ...")
        run_script_result = subprocess.run(['Rscript','-e', f"library({pkg_i})"], capture_output=True)
        if run_script_result.returncode != 0:
            print(f"Installing {pkg_i}")
            run_script_result = subprocess.run(['Rscript','-e', f"install.packages({pkg_i}, repos='https://cloud.r-project.org')"], capture_output=True)
        else:
            print(f"Library {pkg_i} already installed")
            
        print(run_script_result.stderr.decode('UTF-8'))



In [ ]:
install_R_package(list(set([L for V in data_file_flow.values() for L in V['libs']])))

INFO:__main__:Library zoo already installed
DEBUG:__main__:
Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


INFO:__main__:Library lubridate already installed
DEBUG:__main__:
Attaching package: ‘lubridate’

The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union


INFO:__main__:Library mgcv already installed
DEBUG:__main__:Loading required package: nlme
This is mgcv 1.9-1. For overview type 'help("mgcv-package")'.

INFO:__main__:Library stringr already installed
DEBUG:__main__:


## Running the scripts

### Checksum calculation

Each script has it's own set of input files and should be run to
update it's output when either it's inputs have changed or it's 
expected output does not exist.

We can check for filechanges using a hashing algorithm, for 
instance MD5 or SHA-256. These are available either in Python
or from the commandline.

Lets look at the commandline version of MD5, on linux this is
`md5sum`, with the input file for the preparation stage:

In [ ]:
print(json.dumps(data_file_flow[list(data_file_flow.keys())[0]]['in'], indent=3))

[
   "Data/Meta_data.csv",
   "Data/Sapflux_Tilia_train.csv",
   "Data/Weather_Tilia_train.csv",
   "Data/Weather_Tilia_pred.csv"
]


md5sum will output hashes to stdout, which `subprocess.run` captures for us

In [ ]:
flow_object_index = 0
input_files = data_file_flow[flow_object_index]['in']

print(json.dumps(input_files, indent=3))

[
   "Data/Meta_data.csv",
   "Data/Sapflux_Tilia_train.csv",
   "Data/Weather_Tilia_train.csv",
   "Data/Weather_Tilia_pred.csv"
]


In [ ]:

md5_encode_result = subprocess.run(
    ['md5sum','-b']+
    input_files, 
    cwd=save_dir,
    capture_output=True)
print(md5_encode_result.stdout.decode('UTF-8'))

4bed61a77505bfd52032591d5c3a6050 *Data/Meta_data.csv
6d705d98caa6618a4a990c3742c16564 *Data/Sapflux_Tilia_train.csv
1232592f9488ce4fbb4ae11ba5be0349 *Data/Weather_Tilia_train.csv
366dac1bf64003d1d08fca6121c036bd *Data/Weather_Tilia_pred.csv



If we want to check the files we run it with the `-c` option and a file with the previously calculated checksums

In [ ]:
script_name = data_file_flow[flow_object_index]['name']

checksum_file = get_save_path(f"input-checksum-{script_name.split('.')[0]}", save_dir)
with open(checksum_file, 'wt') as cf:
    cf.write(md5_encode_result.stdout.decode('UTF-8'))

In [ ]:
md5_check_result = subprocess.run(
    ['md5sum', '-c', checksum_file], 
    cwd=save_dir,
    capture_output=True)
print(md5_check_result.stdout.decode('UTF-8'))
print(f"Run returned code {md5_check_result.returncode}")
if md5_check_result.returncode:
    print(md5_check_result.stderr.decode('UTF-8'))

Data/Meta_data.csv: OK
Data/Sapflux_Tilia_train.csv: OK
Data/Weather_Tilia_train.csv: OK
Data/Weather_Tilia_pred.csv: OK

Run returned code 0


Had there been a change to a file it would have looked like

In [ ]:
md5_check_result = subprocess.run(
    ['md5sum', '-c', checksum_file+'-modified'], 
    cwd=save_dir,
    capture_output=True)
print(md5_check_result.stdout.decode('UTF-8'))
print(f"Run returned code {md5_check_result.returncode}")


Run returned code 1


We don't really need specifics, only the return code will
do for our purpose.

### Checking files


#### Generating names

In [ ]:
#| exports

read_chunk_size = 1024 * 32
def calc_hash_from_flowobject(flow_object:dict)->str:
    '''Calculate a unique hash for a given flow object'''
    return hashlib.md5(repr(flow_object).encode('UTF-8')).hexdigest()

def calc_hash_from_files(files:list, save_dir:str)->str:
    '''Calculate hash from the contents of the input files'''
    hashobj = hashlib.md5()

    # iterate over files 
    for data_file in files:
        full_name = os.path.join(save_dir, data_file)
        if not os.path.isfile(full_name):
            continue
        
        with open(full_name, 'rb') as f:
            # loop till the end of the file
            while True:
                # read only 1024 bytes at a time
                chunk = f.read(read_chunk_size)
                if not chunk:
                    break
                
                hashobj.update(chunk)
        
    return hashobj.hexdigest()

def calc_hash_from_input_files(flow_object:dict, save_dir:str)->str:
    '''Calculate hash from the contents of the input files for a given flow object'''
    return calc_hash_from_files(flow_object['in'], save_dir)

def calc_hash_from_data_files(flow_object:dict, save_dir:str)->str:
    '''Calculate hash from the contents of the input files for a given flow object'''
    return calc_hash_from_files(flow_object['in'] + flow_object['out'], save_dir)


In [ ]:
calc_hash_from_flowobject(data_file_flow[flow_object_index])

'da4b2413f6a22c19a8a7823e6564e746'

In [ ]:
calc_hash_from_input_files(data_file_flow[flow_object_index], save_dir)

'32095cd16a83a2c63f1ab51a58ed96c9'

In [ ]:
calc_hash_from_data_files(data_file_flow[flow_object_index], save_dir)

'65e6d36ac0c8aadb1fb4ee48d4ff88f3'

#### Inputs

In [ ]:
#| exports
def check_script_inputs(flow_object:dict, save_dir:str)->bool:
    """ 
    Check if the input files for a script are up-to-date, returns True if up-to-date.
    """

    checksum_file = get_save_path(f"input-checksum-{calc_hash_from_flowobject(flow_object)}", save_dir)
    md5_check_result = subprocess.run(
        ['md5sum', '-c', checksum_file], 
        cwd=save_dir,
        capture_output=True)
    
    return int(md5_check_result.returncode) == 0

In [ ]:
check_script_inputs(data_file_flow[0], save_dir)

True

#### Outputs
The output is easily checked for existence with `isfile`.

In [ ]:
#| exports
def check_script_output(flow_object:dict, save_dir:str)->bool:
    """ 
    Check if the output files for a script exist, returns True if they all exist.
    """

    return all([
        os.path.isfile(get_save_path(F, save_dir)) 
        for F in flow_object['out']
    ])

In [ ]:
check_script_output(data_file_flow[0], save_dir)

True

#### Generating the checksum file

In [ ]:
#| exports

def generate_checksum_file(flow_object:dict, save_dir:str)->bool:
    """Generates the checksum file for a given flow object"""

    input_files = flow_object['in']
    md5_encode_result = subprocess.run(
        ['md5sum','-b']+
        input_files, 
        cwd=save_dir,
        capture_output=True)
    
    checksum_file = get_save_path(f"input-checksum-{calc_hash_from_flowobject(flow_object)}", save_dir)
    with open(checksum_file, 'wt') as cf:
        cf.write(md5_encode_result.stdout.decode('UTF-8'))

    return md5_encode_result.returncode == 0 and check_script_inputs(flow_object, save_dir)

In [ ]:
generate_checksum_file(data_file_flow[0], save_dir)

True

#### Running once

We don't need and don't _want't_ to run the script if more then once. This 
is not a problem when a script has finished and updated the checksum file, 
but we also want to prevent near simultaneous runs in a multiprocessing 
environment.

We'll use file locking from fcntl directly


In [ ]:

# Use fcntl file locking to prevent multiple processes from running the same code at the same time.
# see https://docs.python.org/3/library/fcntl.html#fcntl.flock

# Create a filename based on input-file contents
lock_file = get_save_path(f"lock-{calc_hash_from_input_files(data_file_flow[0], save_dir)}", save_dir)


Now we use `fcntl.flock` with flags `fcntl.LOCK_EX | fcntl.LOCK_NB` to lock the file for exclusive access, while
an exception is thrown if it's already locked.


In [ ]:
with open(lock_file, 'wt') as cf:
    fcntl.flock(cf, fcntl.LOCK_EX | fcntl.LOCK_NB)
    with open(lock_file, 'wt') as cf2:
        try:
            fcntl.flock(cf2, fcntl.LOCK_EX | fcntl.LOCK_NB)
        except BlockingIOError as locked_error:
            print(locked_error)

[Errno 11] Resource temporarily unavailable


The locks are removed when the file is closed, how convenient

In [ ]:
with open(lock_file, 'wt') as cf:
    fcntl.flock(cf, fcntl.LOCK_EX | fcntl.LOCK_NB)
with open(lock_file, 'wt') as cf:
    fcntl.flock(cf, fcntl.LOCK_EX | fcntl.LOCK_NB)


## Putting it together

### Synchroneous

We need to run a script when either any of it's inputs have changed or any 
of it's outputs do not exist. Return True if a follow-up script should be 
executed, False if nothing changed or executing the script failed.


In [ ]:
#| exports

def run_rscript_wait(flow_object, assets_dir:str, save_dir:str):
    """ Run a script in R 
        args:
            flow_object: dict of flow object
        returns:
            bool: True if a follow-up script might need to be run, False if not

    """
    syslog.debug(f"Running script {flow_object['name']}")
    # Check if output exists and inputs have not changed and return False if 
    # output exists and inputs have not changed
    if check_script_output(flow_object, save_dir) and check_script_inputs(flow_object, save_dir):
        return True
    
    # Create the lock file
    lock_file = get_save_path(f"lock-{calc_hash_from_flowobject(flow_object)}", save_dir)
    with open(lock_file, 'wt') as cf:
        try:
            syslog.debug(f"Locking {lock_file}")
            # Get exclusive lock on the file, is released on file close
            fcntl.flock(cf, fcntl.LOCK_EX | fcntl.LOCK_NB)

            # run the script
            run_script_result = subprocess.run(
                ['Rscript', '--vanilla', get_asset_path(flow_object['name'], assets_dir)],
                cwd=save_dir,
                capture_output=True
            )
            
            # check the return code
            if run_script_result.returncode:
                cf.write(f"Run returned code {run_script_result.returncode}\n")
                cf.write('STDOUT------------\n', run_script_result.stdout.decode('UTF-8'),'\n')
                cf.write('STDERR------------\n', run_script_result.stderr.decode('UTF-8'),'\n')
                return False

        except BlockingIOError as locked_error:
            syslog.debug(locked_error)
            return False

    
    # check the output and generate the checksum file
    return check_script_output(flow_object, save_dir) and generate_checksum_file(flow_object, save_dir)
    

In [ ]:
run_rscript_wait(data_file_flow[0], assets_dir, save_dir) 

DEBUG:__main__:Running script Data_preparation.R


True

In [ ]:
run_rscript_wait(data_file_flow[1], assets_dir, save_dir)

DEBUG:__main__:Running script Prediction_part1.R


True

In [ ]:
run_rscript_wait(data_file_flow[2], assets_dir, save_dir)

DEBUG:__main__:Running script Prediction_part2.R


True

In [ ]:
def clear_results(flow_object :dict, save_dir:str):
    """Clear the results of a given flow object"""
    for fname in flow_object['out']:
        os.remove(get_save_path(fname, save_dir))

In [ ]:
clear_results(data_file_flow[0], save_dir)

### Asynchronous

In the API we can not wait for a script to finish and we'll use Popen instead. This
means we'll have to keep track of the process.

In [ ]:
import typing

In [ ]:
#| exports
RScriptProcess = namedtuple('RScriptProcess', ['lock_file', 'popen'])

def run_rscript_nowait(flow_object, assets_dir:str, save_dir:str) -> RScriptProcess:
    """ Run a script in R 
        args:
            flow_object: dict of flow object
            assets_dir: path to the assets directory
            save_dir: path to the save directory
        returns:
            RScriptProcess: Popen container object for the script
    """
    
    syslog.debug(f"Starting script {flow_object['name']}")

    # check lockfile ---------------------------------------------------------------
    lock_name = calc_hash_from_flowobject(flow_object)
    if run_rscript_nowait.lock_objects.get(lock_name): 
        lock_object = run_rscript_nowait.lock_objects[lock_name]
        if not lock_object.lock_file.closed:
            syslog.debug(f"Lockfile is open for {flow_object['name']} ({lock_name})")
            
            if lock_object.popen is not None:
                syslog.debug(f"No process running for {flow_object['name']} ({lock_name})")
            elif lock_object.popen.poll() is None:
                syslog.debug(f"Script is still running for {flow_object['name']} ({lock_name})")
                return lock_object
            else:
                syslog.debug(f"Script has finished for {flow_object['name']} ({lock_name})")
                # since poll return not-None the script has finished so close the lockfile
                lock_object.lock_file.close()


    # Check if output exists and inputs have not changed and return False if 
    # output exists and inputs have not changed
    if check_script_output(flow_object, save_dir) and check_script_inputs(flow_object, save_dir):
        syslog.debug(f"Output and inputs are up-to-date for {flow_object['name']}")
        return run_rscript_nowait.lock_objects.get(lock_name)

    # Create the lock file -----------------------------------------------------------
    syslog.debug(f"Creating lockfile for {flow_object['name']} ({lock_name})")
    cf = open(get_save_path(f"lock-{lock_name}", save_dir), 'wt')
    
    try:
        # Set lock on lockfile
        fcntl.flock(cf, fcntl.LOCK_EX | fcntl.LOCK_NB)

        # run the script
        popen = subprocess.Popen(
            ['Rscript', '--vanilla', get_asset_path(flow_object['name'], assets_dir)],
            cwd=save_dir,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            encoding='UTF-8',
        )

        run_rscript_nowait.lock_objects[lock_name] =  RScriptProcess(cf, popen)
            
    except BlockingIOError as locked_error:
        cf.close()
        syslog.error(locked_error)

    syslog.debug(f"Done with {flow_object['name']}")

    return run_rscript_nowait.lock_objects.get(lock_name)

run_rscript_nowait.lock_objects = {}

In [ ]:
run_result = run_rscript_nowait(data_file_flow[0], assets_dir, save_dir) 

DEBUG:__main__:Starting script Data_preparation.R
DEBUG:__main__:Creating lockfile for Data_preparation.R (da4b2413f6a22c19a8a7823e6564e746)
DEBUG:__main__:Done with Data_preparation.R


In [ ]:
run_result

RScriptProcess(lock_file=<_io.TextIOWrapper name='/home/fenke/repos/corebridge/nbs/saves/rscript/lock-da4b2413f6a22c19a8a7823e6564e746' mode='wt' encoding='UTF-8'>, popen=<Popen: returncode: None args: ['Rscript', '--vanilla', '/home/fenke/repos/c...>)

In [ ]:
run_result.popen.poll()

In [ ]:
run_result.popen.returncode

In [ ]:
run_result.popen.stdout.read()

''

In [ ]:
if run_result.popen.poll() is not None:
    print(f"Closing lockfile {run_result.lock_file.name}")
    run_result.lock_file.close()

Closing lockfile /home/fenke/repos/corebridge/nbs/saves/rscript/lock-da4b2413f6a22c19a8a7823e6564e746


In [ ]:
test = run_rscript_nowait(data_file_flow[0], assets_dir, save_dir) 

DEBUG:__main__:Starting script Data_preparation.R
DEBUG:__main__:Output and inputs are up-to-date for Data_preparation.R


In [ ]:
test

RScriptProcess(lock_file=<_io.TextIOWrapper name='/home/fenke/repos/corebridge/nbs/saves/rscript/lock-da4b2413f6a22c19a8a7823e6564e746' mode='wt' encoding='UTF-8'>, popen=<Popen: returncode: 0 args: ['Rscript', '--vanilla', '/home/fenke/repos/core...>)

In [ ]:
test.popen.poll()

0

### AICore module class

In [ ]:
class AICoreRModule(AICoreModule):
    pass

### References

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()